In [ ]:
import torch
import torchvision
import cv2
import numpy as np
import json
from PIL import Image

import time

In [10]:
model = torchvision.models.detection.ssd300_vgg16(pretrained=True)
model.eval()

/Users/alekpak/face_recognition_system/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/alekpak/face_recognition_system/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=

In [ ]:
def detect_people(image):
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
    ])
    
    image_tensor = transform(image).float()
    image_tensor = image_tensor.unsqueeze(0)

    with torch.no_grad():
        detections = model(image_tensor)

    return detections[0]


In [22]:
import ffmpeg
import numpy as np
import cv2

input_video_path = "../data/videos/input_task.mp4"
output_video_path = "output_video.mp4"

# Получаем информацию о видео
probe = ffmpeg.probe(input_video_path)
video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
frame_width = int(video_info['width'])
frame_height = int(video_info['height'])
fps = int(video_info['r_frame_rate'].split('/')[0])

# Создаем процесс ffmpeg для чтения видео
process1 = (
    ffmpeg
    .input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    # .run(quiet=True)
    .run_async(pipe_stdout=True)
)

# Создаем процесс ffmpeg для записи видео
process2 = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='rgb24', s=f'{frame_width}x{frame_height}', r=fps)
    .output(output_video_path, pix_fmt='yuv420p')
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

results = []
frame_number = 0  # Счетчик кадров

while True:
    # Читаем кадр из видео
    in_bytes = process1.stdout.read(frame_width * frame_height * 3)
    if not in_bytes:
        break

    # Преобразуем байты в numpy массив
    frame = (
        np
        .frombuffer(in_bytes, np.uint8)
        .reshape([frame_height, frame_width, 3])
    )
    detections = detect_people((frame / 255.0).astype(np.float32))

    # Рассчитываем временную метку (timestamp)
    timestamp = frame_number / fps  # Время в секундах
    # Извлекаем информацию о детекциях
    detection_info = []
    for i in range(len(detections['boxes'])):
        box = detections['boxes'][i].numpy().tolist()  # Bounding box
        score = detections['scores'][i].item()  # Confidence score
        class_id = detections['labels'][i].item()  # Class ID

        detection_info.append({
            "box": box,
            "score": score,
            "class_id": class_id
        })

    # Добавляем результаты в список
    results.append({
        "timestamp": timestamp,
        "detections": detection_info
    })
    # print(f"Frame {frame_number}, Timestamp: {timestamp:.2f} sec")

    # # Добавляем временную метку на кадр
    # frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Конвертируем RGB в BGR для OpenCV
    # cv2.putText(frame_bgr, f"Time: {timestamp:.2f} sec", (10, 30),
    #             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # # Конвертируем обратно в RGB для записи
    # frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    # Записываем кадр в выходное видео
    process2.stdin.write(frame.tobytes())

    frame_number += 1  # Увеличиваем счетчик кадров

# Закрываем процессы
process1.stdout.close()
process2.stdin.close()
process1.wait()
process2.wait()

with open("output.json", "w") as json_file:
    json.dump(results, json_file, indent=4)

print(f"Результаты сохранены в {"output.json"}")


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

KeyboardInterrupt: 

In [2]:
import ffmpeg
import numpy as np
import cv2

input_video_path = "../data/videos/1080p_Видео_от_Записи_для_аналитики (1).mp4"
output_video_path = "output_video.mp4"

# Получаем информацию о видео
probe = ffmpeg.probe(input_video_path)
video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
frame_width = int(video_info['width'])
frame_height = int(video_info['height'])
fps = int(video_info['r_frame_rate'].split('/')[0])

# Создаем процесс ffmpeg для чтения видео
process1 = (
    ffmpeg
    .input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    # .run(quiet=True)
    .run_async(pipe_stdout=True)
)

# Создаем процесс ffmpeg для записи видео
process2 = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='rgb24', s=f'{frame_width}x{frame_height}', r=fps)
    .output(output_video_path, pix_fmt='yuv420p')
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

frame_number = 0  # Счетчик кадров

while True:
    # Читаем кадр из видео
    in_bytes = process1.stdout.read(frame_width * frame_height * 3)
    if not in_bytes:
        break

    # Преобразуем байты в numpy массив
    frame = (
        np
        .frombuffer(in_bytes, np.uint8)
        .reshape([frame_height, frame_width, 3])
    )

    # Рассчитываем временную метку (timestamp)
    timestamp = frame_number / fps  # Время в секундах
    # print(f"Frame {frame_number}, Timestamp: {timestamp:.2f} sec")

    # Добавляем временную метку на кадр
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Конвертируем RGB в BGR для OpenCV
    cv2.putText(frame_bgr, f"Time: {timestamp:.2f} sec", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Конвертируем обратно в RGB для записи
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    # Записываем кадр в выходное видео
    process2.stdin.write(frame_rgb.tobytes())

    frame_number += 1  # Увеличиваем счетчик кадров

# Закрываем процессы
process1.stdout.close()
process2.stdin.close()
process1.wait()
process2.wait()

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

KeyboardInterrupt: 

In [ ]:
import ffmpeg
import numpy as np
import torch
import torchvision
import json
from PIL import Image

# Загрузите модель (замените на вашу модель)
model = torchvision.models.detection.ssd300_vgg16(pretrained=True)
model.eval()

def detect_people(image):
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
    ])
    
    image_tensor = transform(image)
    image_tensor = image_tensor.float()  # Преобразуем в float32
    image_tensor = image_tensor.unsqueeze(0)

    with torch.no_grad():
        detections = model(image_tensor)

    return detections[0]

input_video_path = "../data/videos/input_task.mp4"
output_video_path = "output_video.mp4"
output_json_path = "detections.json"  # Файл для записи результатов

# Получаем информацию о видео
probe = ffmpeg.probe(input_video_path)
video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
frame_width = int(video_info['width'])
frame_height = int(video_info['height'])
fps = int(video_info['r_frame_rate'].split('/')[0])

# Создаем процесс ffmpeg для чтения видео
process1 = (
    ffmpeg
    .input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run_async(pipe_stdout=True)
)

# Список для хранения результатов
results = []

frames_skip = 5
frame_number = 0

while True:
    # Читаем кадр из видео
    in_bytes = process1.stdout.read(frame_width * frame_height * 3)
    if not in_bytes:
        break

    # Пропускаем каждый второй кадр
    if frame_number % frames_skip != 0:
        frame_number += 1
        continue  # Пропускаем нечетные кадры

    # Преобразуем байты в numpy массив
    frame = (
        np
        .frombuffer(in_bytes, np.uint8)
        .reshape([frame_height, frame_width, 3])
    )

    # Нормализуем изображение и преобразуем в float32
    frame_normalized = (frame / 255.0).astype(np.float32)

    # Детекция людей
    detections = detect_people(frame_normalized)

    # Рассчитываем временную метку
    timestamp = frame_number / fps

    # Извлекаем информацию о детекциях
    detection_info = []
    for i in range(len(detections['boxes'])):
        box = detections['boxes'][i].numpy().tolist()  # Bounding box
        score = detections['scores'][i].item()  # Confidence score
        class_id = detections['labels'][i].item()  # Class ID

        detection_info.append({
            "box": box,
            "score": score,
            "class_id": class_id
        })

    # Добавляем результаты в список
    results.append({
        "timestamp": timestamp,
        "detections": detection_info
    })

    frame_number += 1

# Закрываем процессы
process1.stdout.close()
process2.wait()

# Записываем результаты в JSON-файл


/Users/alekpak/face_recognition_system/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/alekpak/face_recognition_system/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enabl

NameError: name 'process2' is not defined

In [2]:
with open(output_json_path, "w") as json_file:
    json.dump(results, json_file, indent=4)

print(f"Результаты сохранены в {output_json_path}")

Результаты сохранены в detections.json
